In [23]:
import numpy as np
import pandas as pd

In [24]:
#-- Now load the packages for plotting the basemap
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap
from itertools import chain

def draw_map(m, scale=0.2):
    #-- draw a shaded-relief image
    m.shadedrelief(scale=scale)
    
    #-- lats and longs are returned as a dictionary
    lats = m.drawparallels(np.linspace(-90, 90, 13))
    lons = m.drawmeridians(np.linspace(-180, 180, 13))

    #-- keys contain the plt.Line2D instances
    lat_lines = chain(*(tup[1][0] for tup in lats.items()))
    lon_lines = chain(*(tup[1][0] for tup in lons.items()))
    all_lines = chain(lat_lines, lon_lines)
    
    #-- cycle through these lines and set the desired style
    for line in all_lines:
        line.set(linestyle='-', alpha=0.3, color='w')

In [25]:
#-- Run this cell to determine the encoding format of the source.
if False:
    import chardet
    rawdata = open("source/tweets_all.csv", "rb").read()
    result = chardet.detect(rawdata)
    charenc = result['encoding']
    print(charenc)

In [26]:
df = pd.read_csv("source/tweets_all.csv",encoding="Windows-1252") 

In [27]:
#-- Let's check the column headers
# type(df)
# df.head(2)
# [col for col in df.columns]

In [33]:
#-- Now get the timestamps
# df['time']=df['created_at'].str.contains('..:..:..', regex=True) # Checks if regex matches
df['time']=df['created_at'].str.extract('(..:..:..)', expand=True)
df['time'].head(3) # Check the top 3 headers

0    09:49:46
1    09:49:49
2    09:49:49
Name: time, dtype: object

In [37]:
#-- Build a clean dataframe with only the columnns that you need
df_subset = df[['time','place_lat','place_lon']]
df_subset.head(5)

,time,place_lat,place_lon
0,09:49:46,40.227928,-74.703982
1,09:49:49,49.145331,-81.073921
2,09:49:49,39.870397,-85.996819
3,09:49:51,48.660176,2.576654
4,09:50:00,8.942483,7.253046


In [66]:
#-- Now sort the tweets by time
df_subset = df_subset.sort_values(by='time')
df_subset.head(5)

,time,place_lat,place_lon
2661,00:00:34,11.022624,76.879092
2662,00:00:52,51.489263,-0.088161
2663,00:02:32,13.845264,100.581740
2664,00:02:36,3.138874,101.686968
2665,00:02:39,14.654146,121.259866


In [107]:
#-- Now lets group the data into time 'bins'. 
#-- First create a datetime rante
df_time = pd.date_range("00:00", "23:59", freq="30min").time
print(df_time[2])

01:00:00


In [133]:
#-- Now loop through df_subset and update a list based on the datetime bins
from itertools import islice
from datetime import datetime

time_bins = []
i=0
for irow in islice(df_subset.itertuples(), 3):
#     print(irow[1]) #-- Checks timestamp
#     print(type(datetime.strptime(irow[1], '%H:%M:%S').time())) #-- Convert to datetime.time format
#     print(type(df_time[2])) #-- Check to df_time entry
#     if datetime.strptime(irow[1], '%H:%M:%S').time() < df_time[2]:
#         print('yay!')
    for idx,time in enumerate(df_time[0:5]):
#         print(df_time[idx])
        chktime = datetime.strptime(irow[1], '%H:%M:%S').time()
        timelim1 = df_time[idx]
        timelim2 = df_time[idx+1]
        if timelim1 <= chktime < timelim2:
            print('yay!')
            break
        else:
            print('no')
    time_bins.append()
#         print(type(time))

#     print(df_subset)
# for i,idx in df_subset.iterrows():
#     print(df_subset['time'])


yay!
Pandas(Index=2661, time='00:00:34', place_lat=11.022624, place_lon=76.879092)
yay!
Pandas(Index=2662, time='00:00:52', place_lat=51.489263, place_lon=-0.088161)
yay!
Pandas(Index=2663, time='00:02:32', place_lat=13.845264000000002, place_lon=100.58174)


In [ ]:
# Now test the visualisation for one
fig1 = plt.figure(facecolor='w')
ax1  = fig1.add_subplot(111)
# fig = plt.figure(figsize=(8, 6), edgecolor='w')
map = Basemap(projection='cyl', resolution=None,
            llcrnrlat=-90, urcrnrlat=90,
            llcrnrlon=-180, urcrnrlon=180, )
draw_map(map)

lon = -135.3318
lat = 57.0799
x,y = map(lon, lat)
map.plot(x, y, 'bo', markersize=24)
 
plt.show()

> Now test an animated style of plotting.

In [ ]:
from matplotlib import animation, rc
from IPython.display import HTML

In [ ]:
fig, ax = plt.subplots()
ax.set_xlim([0,2])
ax.set_ylim([-2,2])

line, = ax.plot([],[],lw=2)

In [ ]:
def init():
    line.set_data([],[])
    return (line,)

In [ ]:
def animate(i):
    x = np.linspace(0,2,1000)
    y = np.sin(2*np.pi*(x-0.01*i))
    line.set_data(x,y)
    return(line,)

In [ ]:
anim = animation.FuncAnimation(fig, animate, init_func=init,
                              frames=100, interval=20,
                              blit=True)

In [ ]:
HTML(anim.to_html5_video())

In [ ]:
HTML(anim.to_jshtml())